#### Context
I was interested in analyzing how NFL offenses have evolved over the modern era. I used www.pro-football-reference.com to find the data that I was interested in analyzing. I used Python BeautifulSoup4 and a selenium Webdriver to pull the html data from the website and load it into a DataFrame for analysis.

##### Content
There is currently data on 672 Teams across 21 seasons. The stats included are regarding team record, passing, rushing, penalties, turnovers, and more.
The file team_stats_2003_2023.csv contains 35 variables outlined below.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

team_stats = pd.read_csv("team_stats_2003_2023.csv")
team_stats.head()

##### note that a lot of EDA needs to be done on the data

In [ ]:
team_stats.info(verbose=True)

In [ ]:
team_stats.isna().sum()

In [ ]:
team_list = team_stats['team'].unique()
no_teams = len(team_list)

print(f"there are {no_teams} teams in the dataset")

In [ ]:
# named the wlp(wins, losses, points to save strength)

wlp = team_stats.groupby(by=['team'])[['wins', 'losses', 'points']].sum()
wlp['total_games'] = wlp['wins'] + wlp['losses']
wlp.sort_values(by='points', ascending=False)


In [ ]:
# rectify the issue with this function: reference to the matplotlib library

def plot(df, col1, *col2, xlabel, ylabel, title):
    plt.bar(height=df.index, x=df[col1],)
    plt.title(title)
    plt.xlabel(xlabel=xlabel)
    plt.ylabel(ylabel=ylabel)
    plt.show()
plot(df=wlp, col1='wins', xlabel='teams', ylabel='wins', title='wins per team')

In [ ]:
team_stats['ties'].isna().sum()

# probably going to drop this because of missing data

#### Pobably use machine learning to create a model that can predict team wins 

In [ ]:
# this is going to involve supervised learning so the sklearn module is going to be used
# using lazypredict module to see the best regression models for running the predictions

from lazypredict.Supervised import LazyRegressor

lazypred = LazyRegressor(regressors='all', ignore_warnings=True)
models, predictions = lazypred.fit()
models

In [ ]:
from sklearn.
from sklearn.
from sklearn.
# incase i need to make an ensemble of models
from sklearn.ensemble
from sklearn.pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix